In [ ]:
import numpy as np 
import pandas as pd

import os
print(os.listdir("../input"))
#print(os.listdir())

In [ ]:
%%time
train = pd.read_csv("../input/train.csv", nrows = 5_000_000) #設定要train的筆數
test = pd.read_csv("../input/test.csv")

#train = pd.read_csv("train.csv", nrows = 100) #設定要train的筆數
#test = pd.read_csv("test.csv")

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train.dtypes

In [ ]:
train.describe()

# Data preprocessing

## Manhattan vector

In [ ]:
def travel_vector_features(df):
    df['abs_diff_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
    df['abs_diff_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()

travel_vector_features(train)
travel_vector_features(test)

In [ ]:
data = [train, test]
for i in data:
    i["key"] = pd.to_datetime(i["key"])
    i["pickup_datetime"] = pd.to_datetime(i["pickup_datetime"])

NaN

In [ ]:
train.isnull().sum()

In [ ]:
train = train.dropna(how = 'any', axis = 'rows')

In [ ]:
test.isnull().sum()

# Natural Constraint

In [ ]:
train = train[(train.abs_diff_longitude < 5.0) & (train.abs_diff_latitude < 5.0)]

In [ ]:
train = train[train.fare_amount>=0]

In [ ]:
train[train['passenger_count']>6] #檢查乘客大於6的，再把他drop掉

In [ ]:
print(test['pickup_longitude'].max(), test['pickup_longitude'].min(), 
      test['pickup_latitude'].max(), test['pickup_latitude'].min())

In [ ]:
train = train[train.pickup_longitude<=-72.986532]
train = train[train.pickup_longitude>=-74.252193]
train = train[train.pickup_latitude<=41.709555]
train = train[train.pickup_latitude>=40.573143]

In [ ]:
print(test['dropoff_longitude'].max(), test['dropoff_longitude'].min(), 
      test['dropoff_latitude'].max(), test['dropoff_latitude'].min())

In [ ]:
train = train[train.dropoff_longitude<=-72.990963]
train = train[train.dropoff_longitude>=-74.263242]
train = train[train.dropoff_latitude<=41.696683]
train = train[train.dropoff_latitude>=40.568973]

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.head()

In [ ]:
test.head()

# Haversine distance
https://www.kaggle.com/madhurisivalenka/cleansing-eda-modelling-lgbm-xgboost-starters

In [ ]:
def haversine_distance(lat1, long1, lat2, long2):
    data = [train, test]
    for i in data:
        R = 6371  #radius of earth in kilometers
        #R = 3959 #radius of earth in miles
        phi1 = np.radians(i[lat1])
        phi2 = np.radians(i[lat2])
    
        delta_phi = np.radians(i[lat2]-i[lat1])
        delta_lambda = np.radians(i[long2]-i[long1])
    
        #a = sin²((φB - φA)/2) + cos φA . cos φB . sin²((λB - λA)/2)
        a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
    
        #c = 2 * atan2( √a, √(1−a) )
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
        #d = R*c
        d = (R * c) #in kilometers
        i['H_Distance'] = d
    return d

In [ ]:
haversine_distance('pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude')

In [ ]:
data = [train,test]
for i in data:
    i['Year'] = i['pickup_datetime'].dt.year
    i['Month'] = i['pickup_datetime'].dt.month
    i['Date'] = i['pickup_datetime'].dt.day
    i['Day of Week'] = i['pickup_datetime'].dt.dayofweek
    i['Hour'] = i['pickup_datetime'].dt.hour

In [ ]:
train.head()

In [ ]:
test.head()

# reverse_geocoder

In [ ]:
import reverse_geocoder as rg

In [ ]:
#train
train_pickup = train[['pickup_latitude', 'pickup_longitude']]
train_dropoff = train[['dropoff_latitude', 'dropoff_longitude']]

pickup_place = [] #array
dropoff_place = []

for i, j in train_pickup.iterrows():
    pickup_place.append((j["pickup_latitude"], j["pickup_longitude"]))
    
for i, j in train_dropoff.iterrows():
    dropoff_place.append((j["dropoff_latitude"], j["dropoff_longitude"]))

pickup_results = rg.search(pickup_place)
dropoff_results = rg.search(dropoff_place)

pur = [pickup_results[i]['name'] for i in range(0, len(pickup_results))]
dor = [dropoff_results[i]['name'] for i in range(0, len(dropoff_results))]

train['pickup_place'] = pd.Series(pur).astype('category').values
train['dropoff_place'] = pd.Series(dor).astype('category').values

In [ ]:
#test
train_pickup = test[['pickup_latitude', 'pickup_longitude']]
train_dropoff = test[['dropoff_latitude', 'dropoff_longitude']]

pickup_place = [] #array
dropoff_place = []

for i, j in train_pickup.iterrows():
    pickup_place.append((j["pickup_latitude"], j["pickup_longitude"]))
    
for i, j in train_dropoff.iterrows():
    dropoff_place.append((j["dropoff_latitude"], j["dropoff_longitude"]))

pickup_results = rg.search(pickup_place)
dropoff_results = rg.search(dropoff_place)

pur = [pickup_results[i]['name'] for i in range(0, len(pickup_results))]
dor = [dropoff_results[i]['name'] for i in range(0, len(dropoff_results))]

test['pickup_place'] = pd.Series(pur).astype('category').values
test['dropoff_place'] = pd.Series(dor).astype('category').values

In [ ]:
print(train.shape)
print(train.pickup_place.unique())
print(train.dropoff_place.unique())
print(test.pickup_place.unique())
print(test.dropoff_place.unique())

In [ ]:
train = train[train.pickup_place.isin(test.pickup_place)]
train = train[train.dropoff_place.isin(test.dropoff_place)]

In [ ]:
print(train.shape)
print(train.pickup_place.unique())
print(train.dropoff_place.unique())
print(test.pickup_place.unique())
print(test.dropoff_place.unique())

# Categorical Data

In [ ]:
traincv = train.iloc[:80_000, :]

In [ ]:
traincv.head()

https://www.kaggle.com/c/home-credit-default-risk/discussion/58950
<br>https://www.kaggle.com/mlisovyi/beware-of-categorical-features-in-lgbm
<br>https://lightgbm.readthedocs.io/en/latest/Features.html#optimal-split-for-categorical-features

In [ ]:
features = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 
                'passenger_count', 'abs_diff_longitude', 'abs_diff_latitude', 'H_Distance', 
                'Year', 'Month', 'Date', 'Day of Week', 'Hour', 'pickup_place', 'dropoff_place']
target = ['fare_amount']

In [ ]:
X_features = train.iloc[:, :][features]
X_target = train.iloc[:, :][target]

X_featurescv = traincv.iloc[:, :][features]
X_targetcv = traincv.iloc[:, :][target]

In [ ]:
y_features = test.iloc[:, :][features]

In [ ]:
X_features.head()

In [ ]:
X_featurescv.head()

In [ ]:
y_features.head()

# One-hot encoding

In [ ]:
#X_features_encoded = pd.get_dummies(X_features, columns=['pickup_place', 'dropoff_place'])
#y_features_encoded = pd.get_dummies(y_features, columns=['pickup_place', 'dropoff_place'])

In [ ]:
#X_set_features = set(X_features_encoded.columns)
#y_set_features = set(y_features_encoded.columns)
#print(len(X_set_features))
#print(len(y_set_features))
#X_set_features == y_set_features

In [ ]:
#y_reindexed = y_features_encoded.reindex(columns = X_features_encoded.columns, fill_value = 0)

In [ ]:
#X_features_encoded.columns == y_reindexed.columns

In [ ]:
#X_features_encoded.head()

In [ ]:
#y_reindexed.head()

# lgb Model
## GridSearchCV

https://www.kaggle.com/garethjns/microsoft-lightgbm-with-parameter-tuning-0-823
<br>https://lightgbm.readthedocs.io/en/latest/Features.html#optimal-split-for-categorical-features

In [ ]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X_features, X_target, test_size=0.3, random_state=0)
train_Xcv, test_Xcv, train_ycv, test_ycv = train_test_split(X_featurescv, X_targetcv, test_size=0.3, random_state=0)

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

In [ ]:
#dataset
lgb_train = lgb.Dataset(train_X, train_y,
#                        categorical_feature=features, 
                        silent=True, 
                        free_raw_data=False)
lgb_eval = lgb.Dataset(test_X, test_y, reference=lgb_train)

#dataset
lgb_traincv = lgb.Dataset(train_Xcv, train_ycv,
#                        categorical_feature=features, 
                        silent=True, 
                        free_raw_data=False)
lgb_evalcv = lgb.Dataset(test_Xcv, test_ycv, reference=lgb_train)

In [ ]:
def lgb_gridmodel(train_X, test_X, train_y, test_y):
    import lightgbm as lgb
    from sklearn.model_selection import GridSearchCV
    #dataset
    lgb_train = lgb.Dataset(train_X, train_y)
    lgb_eval = lgb.Dataset(test_X, test_y, reference=lgb_train)
    #基本配備，初始參數boosting_type, objective, metric
    global params
    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'rmse',
    }
    #cv
    min_merror = float('Inf')
    global best_params
    best_params = {}
    #num_leaves, max_depth
    for num_leaves in range(20,200,5):
        for max_depth in range(3,8,1):
            params['num_leaves'] = num_leaves
            params['max_depth'] = max_depth
            cv_results = lgb.cv(
                params,
                lgb_train,
                seed=2019,
                stratified=False, 
                nfold=5,
                metrics=['rmse'],
                early_stopping_rounds=10,
                verbose_eval=50
            )
        
            mean_merror = pd.Series(cv_results['rmse-mean']).min()
            boost_rounds = pd.Series(cv_results['rmse-mean']).argmin()
            
            if mean_merror < min_merror:
                min_merror = mean_merror
                best_params['num_leaves'] = num_leaves
                best_params['max_depth'] = max_depth
    params['num_leaves'] = best_params['num_leaves']
    params['max_depth'] = best_params['max_depth']
    
    #max_bin, min_child_samples, min_child_weight(沒調)
    for max_bin in range(5,255,5):
        for min_data_in_leaf in range(10,200,5):
            for min_child_weight in [0.001, 0.002, 0.003, 0.004, 0.005]:
                params['max_bin'] = max_bin
                params['min_data_in_leaf'] = min_data_in_leaf
                params['min_child_weight'] = min_child_weight
                cv_results = lgb.cv(
                    params,
                    lgb_train,
                    seed=42,
                    stratified=False,
                    nfold=5,
                    early_stopping_rounds=3,
                    verbose_eval=50
                )
                mean_merror = pd.Series(cv_results['rmse-mean']).min()
                boost_rounds = pd.Series(cv_results['rmse-mean']).argmin()

                if mean_merror < min_merror:
                    min_merror = mean_merror
                    best_params['max_bin'] = max_bin
                    best_params['min_data_in_leaf'] = min_data_in_leaf
                    best_params['min_child_weight'] = min_child_weight
    params['max_bin'] = best_params['max_bin']
    params['min_data_in_leaf'] = best_params['min_data_in_leaf']
    params['min_child_weight'] = best_params['min_child_weight']
    
    #feature_fraction, bagging_fraction, bagging_freq
    for feature_fraction in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
        for bagging_fraction in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
            for bagging_freq in range(0,50,5):
                params['feature_fraction'] = feature_fraction
                params['bagging_fraction'] = bagging_fraction
                params['bagging_freq'] = bagging_freq
                cv_results = lgb.cv(
                    params,
                    lgb_train,
                    seed=42,
                    stratified=False,
                    nfold=5,
                    metrics=['rmse'],
                    early_stopping_rounds=3,
                    verbose_eval=50
                )
                mean_merror = pd.Series(cv_results['rmse-mean']).min()
                boost_rounds = pd.Series(cv_results['rmse-mean']).argmin()

                if mean_merror < min_merror:
                    min_merror = mean_merror
                    best_params['feature_fraction'] = feature_fraction
                    best_params['bagging_fraction'] = bagging_fraction
                    best_params['bagging_freq'] = bagging_freq
    params['feature_fraction'] = best_params['feature_fraction']
    params['bagging_fraction'] = best_params['bagging_fraction']
    params['bagging_freq'] = best_params['bagging_freq']
    
    #lambda_l1, lambda_l2, min_split_gain
    for lambda_l1 in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
        for lambda_l2 in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
            for min_split_gain in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
                params['lambda_l1'] = lambda_l1
                params['lambda_l2'] = lambda_l2
                params['min_split_gain'] = min_split_gain
                cv_results = lgb.cv(
                    params,
                    lgb_train,
                    seed=42,
                    stratified=False,
                    nfold=5,
                    metrics=['rmse'],
                    early_stopping_rounds=3,
                    verbose_eval=50
                )
                mean_merror = pd.Series(cv_results['rmse-mean']).min()
                boost_rounds = pd.Series(cv_results['rmse-mean']).argmin()
            
                if mean_merror < min_merror:
                    min_merror = mean_merror
                    best_params['lambda_l1'] = lambda_l1
                    best_params['lambda_l2'] = lambda_l2
                    best_params['min_split_gain'] = min_split_gain
    params['lambda_l1'] = best_params['lambda_l1']
    params['lambda_l2'] = best_params['lambda_l2']
    params['min_split_gain'] = best_params['min_split_gain']
    return best_params

In [ ]:
%%time
lgb_gridmodel(train_Xcv, test_Xcv, train_ycv, test_ycv)

In [ ]:
best_params

In [ ]:
params['learning_rate']=0.4
params

In [ ]:
gbm = lgb.train(
    params,                     
    lgb_train,                  
    valid_sets=lgb_eval,       
    num_boost_round=300,    
    early_stopping_rounds=50
)

In [ ]:
y_pred_lgb = gbm.predict(y_features, num_iteration = gbm.best_iteration)

In [ ]:
#feature importance
%matplotlib inline
import matplotlib.pyplot as plt

ax = lgb.plot_importance(gbm)

In [ ]:
submission = pd.DataFrame()
submission['key'] = test.key
submission['fare_amount'] = y_pred_lgb
submission.to_csv('submission_lgbgrid.csv', index = False)

In [ ]:
ss = pd.read_csv('../input/sample_submission.csv')
#ss = pd.read_csv('sample_submission.csv')
ss.loc[:, 'fare_amount'] = y_pred_lgb

ss.to_csv('submission_lgbgrid.csv', index=False)

# top

In [ ]:
top_params = {
        'boosting_type':'gbdt',
        'objective': 'regression',
        'nthread': -1,
        'verbose': 0,
        'num_leaves': 31,
        'learning_rate': 0.4,
        'max_depth': -1,
        'subsample': 0.8,
        'subsample_freq': 1,
        'colsample_bytree': 0.6,
        'reg_aplha': 1,
        'reg_lambda': 0.001,
        'metric': 'rmse',
        'min_split_gain': 0.5,
        'min_child_weight': 1,
        'min_child_samples': 10,
        'scale_pos_weight':1,
        'verbose':0
    }

In [ ]:
gbmtop = lgb.train(
    top_params,                     
    lgb_train,                  
    valid_sets=lgb_eval,       
    num_boost_round=300,    
    early_stopping_rounds=50
)

In [ ]:
y_pred_lgbtop = gbmtop.predict(y_features, num_iteration = gbm.best_iteration)

In [ ]:
#feature importance
%matplotlib inline
import matplotlib.pyplot as plt

ax = lgb.plot_importance(gbmtop)

In [ ]:
submission = pd.DataFrame()
submission['key'] = test.key
submission['fare_amount'] = y_pred_lgbtop
submission.to_csv('submission_lgbtop.csv', index = False)

In [ ]:
ss = pd.read_csv('../input/sample_submission.csv')
#ss = pd.read_csv('sample_submission.csv')
ss.loc[:, 'fare_amount'] = y_pred_lgbtop

ss.to_csv('submission_lgbtop.csv', index=False)